In [ ]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

# Laser Shading By adjusting focus on the Z-axis

# Code:

In [2]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # [mm]
BlockLength = 2 * BlockHeight  # [mm]
BlockThickness = 19.0  # [mm]


In [3]:
def init(**kwargs):
    program = GCode.GCode(**kwargs)
    program.G0(F=500)
    program.G1(F=300)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    program.M5()
    return program


def end(**kwargs):
    program = GCode.GCode(**kwargs)
    program.M5()
    return program


In [4]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])


In [6]:
cnc.reset()


(0, ['ok', '', "Grbl 1.1f ['$' for help]", "[MSG:'$H'|'$X' to unlock]", 'ok'])

In [9]:
if "Alarm" in cnc.status:
    print("Homing.")
    cnc.home()


In [10]:
init(machine=cnc).run()


In [11]:
BlockHeight


89.0

In [12]:
BlockLength


178.0

In [24]:
vgap = 5
vs = np.arange(vgap, BlockHeight, vgap)
vs


array([ 5., 10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60., 65.,
       70., 75., 80., 85.])

In [25]:
len(vs)


17

In [26]:
hgap = 10
hs = np.arange(0, BlockLength, hgap)
hs


array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
       110., 120., 130., 140., 150., 160., 170.])

In [27]:
ztravel = -28


In [95]:
zs = np.linspace(start=ztravel, stop=0, num=len(hs) - 1, endpoint=True)
zs


array([-28.  , -26.25, -24.5 , -22.75, -21.  , -19.25, -17.5 , -15.75,
       -14.  , -12.25, -10.5 ,  -8.75,  -7.  ,  -5.25,  -3.5 ,  -1.75,
         0.  ])

In [96]:
len(vs)


17

In [97]:
len(vs) // 3


5

In [98]:
powers = np.linspace(100, 255, 5, endpoint=True)
powers


array([100.  , 138.75, 177.5 , 216.25, 255.  ])

In [99]:
zs


array([-28.  , -26.25, -24.5 , -22.75, -21.  , -19.25, -17.5 , -15.75,
       -14.  , -12.25, -10.5 ,  -8.75,  -7.  ,  -5.25,  -3.5 ,  -1.75,
         0.  ])

In [136]:
program = GCode.GCode()
program.G0(F=500)
program.G1(F=300)
program.M3(S=1)

feeds = [100, 200, 300]

for idx1, feed in enumerate(feeds):  # [mm/min]
    for idx2, power in enumerate(powers):
        v_idx = idx1 * len(feeds) + idx2
        Y = vs[v_idx]
        program.M5()
        program.G0(X=hs[0], Y=Y)
        program.M4(S=power)
        for idx3, z in enumerate(zs):
            x0 = hs[idx3]
            xf = hs[idx3 + 1]
            program.G0(Z=z, X=x0)
            program.G1(X=xf, Y=Y, F=feed)
            break
        break
    break


In [137]:
program


<GCode>[cmds=8]